## Data Loaders

This notebook demonstrates the use of **graph loader** in `pyTigerGraph`. The job of a data loader is to pull data from the TigerGraph database. Currently, the following data loaders are provided:
* EdgeLoader, which returns batches of edges.
* VertexLoader, which returns batches of vertices.
* GraphLoader, which returns randomly sampled (probably disconnected) subgraphs in pandas `dataframe`, `PyG` or `DGL` format.
* NeighborLoader, which returns subgraphs using neighbor sampling in `dataframe`, `PyG` or `DGL` format.
* EdgeNeighborLoader, which returns subgraphs using neighbor sampling from edges in `dataframe`, `PyG` or `DGL` format.

Every data loader above can either get all the batches as a HTTP response (default) or stream every batch through Kafka. The former mechanism is good for testing with small graphs and it is fast, but it subjects to a data size limit of 2GB. For large graphs, the HTTP channel will likely fail due to size limit and network connectivity issues. Streaming via Kafka is offered for data robustness and scalability. Also, Kafka excels at multi-consumer use cases, and it is efficient for model search or hyperparameter tuning when there are multiiple consumers of the same data. 

The data loaders support both homogeneous and heterogenous graphs. By default, they load from all vertex and edge types and treat the graph as a homogeneous graph. But they also allow users to specify what vertex and edge types to load from and what attributes to load from each type. This way users will get heterogeneous graph outputs.



### Connection to Database

The `TigerGraphConnection` class represents a connection to the TigerGraph database. Under the hood, it stores the necessary information to communicate with the database. It is able to perform quite a few database tasks. Please see its [documentation](https://docs.tigergraph.com/pytigergraph/current/intro/) for details.

In [23]:
from pyTigerGraph import TigerGraphConnection
import json

# Read in DB configs
with open('../../config.json', "r") as config_file:
    config = json.load(config_file)
    
conn = TigerGraphConnection(
    host=config["host"],
    username=config["username"],
    password=config["password"]
)

from pyTigerGraph.datasets import Datasets

dataset = Datasets("Cora")

conn.ingestDataset(dataset, getToken=config["getToken"])

from pyTigerGraph.visualization import drawSchema

drawSchema(conn.getSchema(force=True))

A folder with name Cora already exists in ./tmp. Skip downloading.
---- Checking database ----
A graph with name Cora already exists in the database. Skip ingestion.
Graph name is set to Cora for this connection.


CytoscapeWidget(cytoscape_layout={'name': 'circle', 'animate': True, 'padding': 1}, cytoscape_style=[{'selecto…

### Neighbor Loader

`NeighborLoader` performs neighbor sampling as introduced in [Inductive Representation Learning on Large Graphs](https://arxiv.org/abs/1706.02216) and returns neighborhood subgraphs. Hence, the subgraphs from this loader are connected. 

Specifically, the loader first chooses `batch_size` number of vertices as seeds, then picks `num_neighbors` number of neighbors of each seed at random, then `num_neighbors` neighbors of each neighbor, and repeat for `num_hops`. This generates one subgraph. As you loop through this data loader, every vertex will at some point be chosen as a seed and you will get the subgraph expanded from the seed. If you want to limit seeds to certain vertices, the boolean attribute provided to `filter_by` will be used to indicate which vertices can be included as seeds.

**Note**: For the first time you initialize the loader on a graph in TigerGraph,
the initialization might take a minute as it installs the corresponding
query to the database and optimizes it. However, the query installation only
needs to be done once, so it will take no time when you initialize the loader
on the same TG graph again.

* First, it can be used as an iterable, which means you can loop through
  it to get every batch of data. If you load all data at once (`num_batches=1`),
  there will be only one batch (of all the data) in the iterator.
* Second, you can access the `data` property of the class directly. If there is
  only one batch of data to load, it will give you the batch directly instead
  of an iterator, which might make more sense in that case. If there are
  multiple batches of data to load, it will return the loader itself.
    
Args:
* v_in_feats (list or dict, optional):
        Vertex attributes to be used as input features. 
        If it is a list, then the attributes
        in the list from all vertex types will be selected. An error will be thrown if
        certain attribute doesn't exist in all vertex types. If it is a dict, keys of the 
        dict are vertex types to be selected, and values are lists of attributes to be 
        selected for each vertex type.
        Only numeric and boolean attributes are allowed. The type of an attribute 
        is automatically determined from the database schema. Defaults to None.
* v_out_labels (list or dict, optional):
        Vertex attributes to be used as labels for prediction. 
        If it is a list, then the attributes
        in the list from all vertex types will be selected. An error will be thrown if
        certain attribute doesn't exist in all vertex types. If it is a dict, keys of the 
        dict are vertex types to be selected, and values are lists of attributes to be 
        selected for each vertex type.
        Only numeric and boolean attributes are allowed. Defaults to None.
* v_extra_feats (list or dict, optional):
        Other attributes to get such as indicators of train/test data. 
        If it is a list, then the attributes
        in the list from all vertex types will be selected. An error will be thrown if
        certain attribute doesn't exist in all vertex types. If it is a dict, keys of the 
        dict are vertex types to be selected, and values are lists of attributes to be 
        selected for each vertex type. 
        All types of attributes are allowed. Defaults to None.
* e_in_feats (list or dict, optional):
        Edge attributes to be used as input features. 
        If it is a list, then the attributes
        in the list from all edge types will be selected. An error will be thrown if
        certain attribute doesn't exist in all edge types. If it is a dict, keys of the 
        dict are edge types to be selected, and values are lists of attributes to be 
        selected for each edge type.
        Only numeric and boolean attributes are allowed. The type of an attribute
        is automatically determined from the database schema. Defaults to None.
* e_out_labels (list or dict, optional):
        Edge attributes to be used as labels for prediction. 
        If it is a list, then the attributes in the list from all edge types will be 
        selected. An error will be thrown if certain attribute doesn't exist in all 
        edge types. If it is a dict, keys of the dict are edge types to be selected, 
        and values are lists of attributes to be selected for each edge type.
        Only numeric and boolean attributes are allowed. Defaults to None.
* e_extra_feats (list or dict, optional):
        Other edge attributes to get such as indicators of train/test data. 
        If it is a list, then the attributes in the list from all edge types will be 
        selected. An error will be thrown if certain attribute doesn't exist in all 
        edge types. If it is a dict, keys of the dict are edge types to be selected, 
        and values are lists of attributes to be selected for each edge type.
        All types of attributes are allowed. Defaults to None.
* batch_size (int, optional):  
        Number of edges in each batch.  
        Defaults to None.  
* num_batches (int, optional):  
        Number of batches to split the edges.  
        Defaults to 1.  
* num_neighbors (int, optional):
        Number of neighbors to sample for each vertex.
        Defaults to 10.
* num_hops (int, optional):
        Number of hops to traverse when sampling neighbors.
        Defaults to 2.
* shuffle (bool, optional):  
        Whether to shuffle the edges before loading data.  
        Defaults to False.  
* filter_by (str, optional):
        A boolean attribute used to indicate which edges are included. Defaults to None.
* output_format (str, optional):
        Format of the output data of the loader. Only
        "dataframe" is supported. Defaults to "dataframe".
* loader_id (str, optional):
        An identifier of the loader which can be any string. It is
        also used as the Kafka topic name. If `None`, a random string will be generated
        for it. Defaults to None.
* buffer_size (int, optional):
        Number of data batches to prefetch and store in memory. Defaults to 4.
* timeout (int, optional):
        Timeout value for GSQL queries, in ms. Defaults to 300000.

# Testcase1: using neighboLoader with callback_fn to get batches of data.(for homogeneous graph)    
## Results: run successfully, data loaded completely

In [25]:
def process_batch(batch):
    return batch
neighbor_loader2 = conn.gds.neighborLoader(
    num_batches=10,
    num_neighbors = 10,
    num_hops =2,
    v_in_feats = ["x"],
    v_out_labels = ["y"],
    v_extra_feats = ["train_mask", "val_mask", "test_mask"],
    e_in_feats=["time"],
    e_out_labels=[],
    e_extra_feats=["is_train", "is_val"],
    output_format = "PyG",
    shuffle=True,
    filter_by=None,
    callback_fn = process_batch
)
for i, batch in enumerate(neighbor_loader2):
    print("----Batch {}----".format(i))
    print(batch)

----Batch 0----
Data(edge_index=[2, 4372], edge_feat=[4372], is_train=[4372], is_val=[4372], x=[1899, 1433], y=[1899], train_mask=[1899], val_mask=[1899], test_mask=[1899], is_seed=[1899])
----Batch 1----
Data(edge_index=[2, 4187], edge_feat=[4187], is_train=[4187], is_val=[4187], x=[1795, 1433], y=[1795], train_mask=[1795], val_mask=[1795], test_mask=[1795], is_seed=[1795])
----Batch 2----
Data(edge_index=[2, 4160], edge_feat=[4160], is_train=[4160], is_val=[4160], x=[1860, 1433], y=[1860], train_mask=[1860], val_mask=[1860], test_mask=[1860], is_seed=[1860])
----Batch 3----
Data(edge_index=[2, 3551], edge_feat=[3551], is_train=[3551], is_val=[3551], x=[1682, 1433], y=[1682], train_mask=[1682], val_mask=[1682], test_mask=[1682], is_seed=[1682])
----Batch 4----
Data(edge_index=[2, 4342], edge_feat=[4342], is_train=[4342], is_val=[4342], x=[1845, 1433], y=[1845], train_mask=[1845], val_mask=[1845], test_mask=[1845], is_seed=[1845])
----Batch 5----
Data(edge_index=[2, 4563], edge_feat=[4

# Testcase2: using neighboLoader with callback_fn to get batchs of data(for heterogeneous graph).  
## case details:using neighboLoader without callback_fn first, then using neighboLoader with the same paprams but set a callback_fn to get part of data.
## Results: run successfully, data loaded completely

Since `Cora` is a homogeneous graph, we will connect to a different graph to demostrate the use case of heterogeneous graphs.

In [26]:
conn.graphname="hetero"

# COMMENT OUT THE LINE BELOW if you are NOT using a graph that requires token authentication
conn.getToken(conn.createSecret())

('6bv96igumojq7ctdikvod3hq2i1itln0', 1674821660, '2023-01-27 12:14:20')

In [27]:
neighbor_loader3 = conn.gds.neighborLoader(
    v_in_feats={"v0": ["x"],
                "v1": ["x"],
                "v2": ["x"]},
    v_out_labels={"v0": ["y"]},
    v_extra_feats={"v0": ["train_mask", "val_mask", "test_mask"]},
    batch_size=16,
    num_neighbors=10,
    num_hops=2,
    shuffle=True,
    output_format="PyG",
    add_self_loop=False,
    loader_id=None,
    buffer_size=4,
)
for i, batch in enumerate(neighbor_loader3):
    print("----Batch {}----".format(i))
    print(batch)

----Batch 0----
HeteroData(
  v1={
    x=[110, 171],
    is_seed=[110]
  },
  v0={
    x=[76, 231],
    y=[76],
    train_mask=[76],
    val_mask=[76],
    test_mask=[76],
    is_seed=[76]
  },
  v2={
    x=[100, 144],
    is_seed=[100]
  },
  (v1, v1v1, v1)={ edge_index=[2, 390] },
  (v1, v1v2, v2)={ edge_index=[2, 427] },
  (v0, v0v0, v0)={ edge_index=[2, 481] },
  (v2, v2v0, v0)={ edge_index=[2, 287] },
  (v2, v2v1, v1)={ edge_index=[2, 300] },
  (v2, v2v2, v2)={ edge_index=[2, 300] }
)
----Batch 1----
HeteroData(
  v1={
    x=[109, 171],
    is_seed=[109]
  },
  v2={
    x=[100, 144],
    is_seed=[100]
  },
  v0={
    x=[76, 231],
    y=[76],
    train_mask=[76],
    val_mask=[76],
    test_mask=[76],
    is_seed=[76]
  },
  (v0, v0v0, v0)={ edge_index=[2, 250] },
  (v1, v1v1, v1)={ edge_index=[2, 349] },
  (v1, v1v2, v2)={ edge_index=[2, 347] },
  (v2, v2v0, v0)={ edge_index=[2, 154] },
  (v2, v2v1, v1)={ edge_index=[2, 195] },
  (v2, v2v2, v2)={ edge_index=[2, 180] }
)
----Batch 

In [28]:
def process_batch(batch):
    return {"v0":batch["v0"]}
loader4 = conn.gds.neighborLoader(
    v_in_feats={"v0": ["x"],
                "v1": ["x"],
                "v2": ["x"]},
    v_out_labels={"v0": ["y"]},
    v_extra_feats={"v0": ["train_mask", "val_mask", "test_mask"]},
    batch_size=16,
    num_neighbors=10,
    num_hops=2,
    shuffle=True,
    output_format="PyG",
    add_self_loop=False,
    loader_id=None,
    buffer_size=4,
    callback_fn=process_batch
)
for i, batch in enumerate(loader4):
    print("----Batch {}----".format(i))
    print(batch)

----Batch 0----
{'v0': {'x': tensor([[-1.2908,  0.9084,  0.6218,  ..., -0.9899,  0.5748,  1.5626],
        [ 1.2661,  1.5683,  0.9176,  ...,  1.3512, -0.4111,  0.2691],
        [-2.0548, -2.5014,  1.5261,  ...,  0.0480, -1.1650, -1.2608],
        ...,
        [-1.2434, -0.6484,  0.6525,  ...,  2.1990, -0.5989,  1.5437],
        [ 0.6978, -0.3825,  0.8817,  ..., -0.7247, -0.2079, -2.5429],
        [-1.1679, -1.1890, -0.6898,  ..., -0.4965,  1.0150,  1.4085]],
       dtype=torch.float64), 'y': tensor([1, 1, 1, 0, 8, 4, 5, 8, 7, 5, 6, 9, 4, 6, 0, 9, 6, 2, 1, 1, 1, 5, 2, 3,
        3, 7, 8, 2, 7, 9, 7, 0, 1, 0, 8, 7, 1, 4, 4, 8, 1, 7, 6, 3, 8, 4, 9, 9,
        6, 9, 9, 8, 4, 9, 9, 5, 4, 1, 7, 8, 1, 6, 4, 4, 7, 7, 7, 7, 1, 4, 1, 2,
        5, 9, 2, 3]), 'train_mask': tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, 

# Testcase3: using neighboLoader with callback_fn to loaddata(via Kafka).  
## Results: run successfully, data loaded completely

**Note**: Kafka streaming function is only available for the Enterprise Edition. You need to activate the Enterprise Edition to use it. 

In [29]:
conn.graphname="Cora"
# COMMENT OUT THE LINE BELOW if you are NOT using a graph that requires token authentication
conn.getToken(conn.createSecret())

('c50hs1j0d853ne0ji1ihbve9e13c10f5', 1674821666, '2023-01-27 12:14:26')

#### Configure Kafka
Set up Kafka here. Once configured, the settings will be shared with all newly created data loaders and no need to set up Kafka for each loader. Please see official [doc](https://docs.tigergraph.com/pytigergraph/current/gds/gds#_configurekafka) for detailed settings.

In [30]:
conn.gds.configureKafka(kafka_address ="your_Kafka_address")

#### Get batches of vertices

In [31]:
def process_batch(batch):
    return batch
neighbor_loader5 = conn.gds.neighborLoader(
    num_batches=10,
    num_neighbors = 10,
    num_hops =2,
    v_in_feats = ["x"],
    v_out_labels = ["y"],
    v_extra_feats = ["train_mask", "val_mask", "test_mask"],
    e_in_feats=["time"],
    e_out_labels=[],
    e_extra_feats=["is_train", "is_val"],
    output_format = "PyG",
    shuffle=True,
    filter_by=None,
    callback_fn = process_batch
)
for i, batch in enumerate(neighbor_loader5):
    print("----Batch {}----".format(i))
    print(batch)

----Batch 0----
Data(edge_index=[2, 3561], edge_feat=[3561], is_train=[3561], is_val=[3561], x=[1634, 1433], y=[1634], train_mask=[1634], val_mask=[1634], test_mask=[1634], is_seed=[1634])
----Batch 1----
Data(edge_index=[2, 4293], edge_feat=[4293], is_train=[4293], is_val=[4293], x=[1870, 1433], y=[1870], train_mask=[1870], val_mask=[1870], test_mask=[1870], is_seed=[1870])
----Batch 2----
Data(edge_index=[2, 3941], edge_feat=[3941], is_train=[3941], is_val=[3941], x=[1782, 1433], y=[1782], train_mask=[1782], val_mask=[1782], test_mask=[1782], is_seed=[1782])
----Batch 3----
Data(edge_index=[2, 4209], edge_feat=[4209], is_train=[4209], is_val=[4209], x=[1802, 1433], y=[1802], train_mask=[1802], val_mask=[1802], test_mask=[1802], is_seed=[1802])
----Batch 4----
Data(edge_index=[2, 4803], edge_feat=[4803], is_train=[4803], is_val=[4803], x=[1936, 1433], y=[1936], train_mask=[1936], val_mask=[1936], test_mask=[1936], is_seed=[1936])
----Batch 5----
Data(edge_index=[2, 4238], edge_feat=[4